# Coursera Applied Data Science Capstone Project

## Table of Contents

<!-- <div class="alert alert-block alert-info" style="margin-top: 20px"> -->

1. <a href="#item1">Introduction</a>

2. <a href="#item2">Data and methods</a>

3. <a href="#item3">Data exploration</a>

4. <a href="#item4">Clustering with K-means</a>

5. <a href="#item5">Visualizing the neighbourhoods</a>    

6. <a href="#item6">Summary</a>

<a id='item1'></a>
# 1. Introduction

Toronto has a large Chinese community. The Toronto metropolitan area has over 600,000 Chinese population accounting for over 10% of total population in 2016. There is a huge demand for Chinese restaurants. This project seeks to visualize the spatial distribution of existing Chinese restaurants in various regions in Toronto and helps **restaurant enterpreneurs** identify locations suitable for new businesses.

<a id='item2'></a>
# 2. Data and methods

The project gathers the locations of existing Chinese restaurants in various regions of Toronto to examine the spatial distribution of Chinese restaurants. It also gathers  demographic information of those regions to identify areas that lack Chinese restaurants relative to the density of Chinese residents.

The restaurant data set used here is gathered from Foursquare APIs. The "search" API is used here and it can return venues that are located at a certain distance range from specific geographic coordinates. It can also be limited to certain categories like "Chinese Restaurant" as specified by the API. Refer to https://developer.foursquare.com/docs/api/venues/search for a more detailed description of the API.

The spatial unit of this analysis is postcode zones. Each postcode zone may contain one or more neighbourhoods. This information is scraped from wikipedia (https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M). 

Demographic information of each postcode is helpful to identify clusters of potential customers. This information if scraped from Statistics Canada (https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/prof/index.cfm?Lang=E). The relevant data include the Chinese population and total population within each postcode zone.

## 2.1 Neighbourhood data collection

### Installing and importing relevant packages 

In [1]:
# !conda install -c anaconda beautifulsoup4 

In [2]:
import requests
import lxml
from bs4 import BeautifulSoup as bs
import pandas as pd

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  34.06 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  36.75 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  18.97 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  22.69 MB/s
Libraries imported.


### Scrape the Toronto wiki page to get post codes and neighbourhoods

In [4]:
wiki_url_m = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page_m = requests.get(wiki_url_m)

In [5]:
html_m = page_m.text
soup_m = bs(html_m,'lxml')
table = soup_m.find('table')
data_m=[]
for th in table.find_all('tr'):
    data_m.append(th.text.strip('\n').split('\n'))

In [6]:
df = pd.DataFrame.from_records(data_m,columns = data_m[0])
df.drop(0,axis=0,inplace=True)

### Drop "not assigned" boroughs

In [7]:
df.drop(df[df.Borough=='Not assigned'].index,axis=0,inplace=True)
# df
df.Neighbourhood[df.Neighbourhood=='Not assigned']=df.Borough[df.Neighbourhood=='Not assigned']
df.reset_index(drop=True,inplace=True)
df.shape

(211, 3)

### Merge neighbourhoods within the same postcode

In [8]:
df1 = df.groupby('Postcode')['Neighbourhood'].apply(lambda x:'%s'%', '.join(x))
df = df.merge(df1.to_frame().reset_index(),on='Postcode',how='left',suffixes=('_left', '_right'))
df.rename(columns={'Neighbourhood_right': 'Neighbourhood'},inplace=True)
df.drop('Neighbourhood_left',axis=1,inplace=True)
df.drop_duplicates(subset='Postcode',inplace=True)
df.shape

(103, 3)

### Download the csv file with postal code coordinates and merge with neighbourhood data

In [9]:
!wget -q -O 'toronto_postcodes.csv' https://cocl.us/Geospatial_data

In [10]:
df_postcodes = pd.read_csv('toronto_postcodes.csv')
df_postcodes.shape

(103, 3)

In [11]:
df1 = df.merge(df_postcodes,left_on='Postcode',right_on='Postal Code',how='left')
df1.drop('Postal Code', axis=1, inplace=True)
df1.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


## 2.2 Demographic data collection

In [12]:
postcode = []
tot = []
total_pop = []
cn_tot_pop = []
for pc in df['Postcode'].values:
    url = 'https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/prof/details/page.cfm?Lang=E&Geo1=FSA&Code1={}&Geo2=PR&Code2=01&Data=Count&SearchType=Begins&SearchPR=01&B1=Ethnic%20origin&TABID=2'.format(pc)
    page = requests.get(url)
    html = page.text
    soup = bs(html,'lxml')
    table = soup.find('table')
    try:
        tot_row = table.find('th',{'id':'L26000'}).text.strip('\r\n').strip(" ").strip('\r\n')
        tot_pop = table.find('th',{'id':'L26000'}).find_parent('tr').find('td',{'headers':'L26000 geo1 total1'}).text
        tot_pop = ''.join(n for n in tot_pop if n.isdigit())
        
        cn_row = table.find('th',{'id':'L26251'}).text.strip('\r\n').strip(" ").strip('\r\n')
        cn_pop = table.find('th',{'id':'L26251'}).find_parent('tr').find('td',{'headers':'L26251 geo1 total1'}).text
        cn_pop = ''.join(n for n in cn_pop if n.isdigit())
        
        postcode.append(pc)
        tot.append(tot_row)
        total_pop.append(tot_pop)
        cn_tot_pop.append(cn_pop)
        
    except:
        pass

In [13]:
df_census = pd.DataFrame({'postcode':postcode,'chinese_pop':cn_tot_pop,'total_pop':total_pop},columns=['postcode','chinese_pop','total_pop'])
df_census

,postcode,chinese_pop,total_pop
0,M3A,3460,34500
1,M4A,815,13865
2,M5A,3955,38930
3,M6A,810,20390
4,M7A,,
5,M9A,1365,34835
6,M1B,5570,65915
7,M3B,3395,13265
8,M4B,985,18600
9,M5B,2290,12055


In [14]:
df_data = df1.merge(df_census,left_on='Postcode',right_on='postcode',how='left')
df_data.drop('postcode', axis=1, inplace=True)

In [15]:
df_data['chinese_pop'] = df_data['chinese_pop'].replace('',0)
df_data['total_pop'] = df_data['total_pop'].replace('',0)
df_data['chinese_pop'] = df_data['chinese_pop'].astype(float)
df_data['total_pop'] = df_data['total_pop'].astype(float)
df_data['chinese_pct'] = df_data['chinese_pop'] / df_data['total_pop']

In [16]:
df_data.sort_values(by='chinese_pct',ascending=False)

,Postcode,Borough,Neighbourhood,Latitude,Longitude,chinese_pop,total_pop,chinese_pct
85,M1V,Scarborough,"Agincourt North, L'Amoreaux East, Milliken, St...",43.815252,-79.284577,36590.0,54410.0,0.672487
78,M1S,Scarborough,Agincourt,43.794200,-79.262029,20105.0,36960.0,0.543966
90,M1W,Scarborough,L'Amoreaux West,43.799525,-79.318389,23040.0,47830.0,0.481706
27,M2H,North York,Hillcrest Village,43.803762,-79.363452,11770.0,24505.0,0.480310
84,M5T,Downtown Toronto,"Chinatown, Grange Park, Kensington Market",43.653206,-79.400049,6215.0,17350.0,0.358213
82,M1T,Scarborough,"Clarks Corners, Sullivan, Tam O'Shanter",43.781638,-79.304302,11450.0,33955.0,0.337211
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,2735.0,8295.0,0.329717
59,M2N,North York,Willowdale South,43.770120,-79.408493,22695.0,75000.0,0.302600
39,M2K,North York,Bayview Village,43.786947,-79.385975,6465.0,23340.0,0.276992
33,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,15970.0,58060.0,0.275060


## 2.3 Restaurant data collection

### Credentials

In [17]:
# The code was removed by Watson Studio for sharing.

### Get rearby venues

In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    category='4bf58dd8d48988d145941735' # chinese restaurants
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            category)
            
        # make the GET request
        results = requests.get(url).json()["response"]['venues']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng'],  
            v['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Restaurant', 
                  'Latitude', 
                  'Longitude', 
                  'Restaurant Category']
    
    return(nearby_venues)

In [20]:
df_restaurants = getNearbyVenues(names=df_data['Neighbourhood'],
                                   latitudes=df_data['Latitude'],
                                   longitudes=df_data['Longitude']
                                  )

Parkwoods
Victoria Village
Harbourfront, Regent Park
Lawrence Heights, Lawrence Manor
Queen's Park
Islington Avenue
Rouge, Malvern
Don Mills North
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Glencairn
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Highland Creek, Rouge Hill, Port Union
Flemingdon Park, Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights
Thorncliffe Park
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
East Birchmount Park, Ionview, Kennedy Park
Bayview Village
CFB Toronto, Downsview East
The D

In [21]:
df_restaurants.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Restaurant,Latitude,Longitude,Restaurant Category
0,"Harbourfront, Regent Park",43.654260,-79.360636,Oriental Taste,43.655304,-79.365312,Chinese Restaurant
1,"Harbourfront, Regent Park",43.654260,-79.360636,Huayu Kitchen,43.654148,-79.357826,Chinese Restaurant
2,"Harbourfront, Regent Park",43.654260,-79.360636,Peking Express,43.656692,-79.365126,Chinese Restaurant
3,"Harbourfront, Regent Park",43.654260,-79.360636,Bamboo Kitchen,43.658620,-79.365881,Chinese Restaurant
4,Queen's Park,43.662301,-79.389494,Manchu Wok,43.659524,-79.390538,Chinese Restaurant


<a id='item3'></a>
## 3. Data exploration

### Number of venues within each neighbourhood (limited to 100)
We see that Rosedale only has 4 venues. Probably should remove it from clustering.

In [22]:
df_restaurants.groupby('Neighbourhood')['Restaurant'].count()

Neighbourhood
Adelaide, King, Richmond                                                                                         28
Agincourt                                                                                                         1
Agincourt North, L'Amoreaux East, Milliken, Steeles East                                                          6
Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown     1
Alderwood, Long Branch                                                                                            1
Bathurst Manor, Downsview North, Wilson Heights                                                                   1
Bayview Village                                                                                                   1
Berczy Park                                                                                                       6
Cabbagetown, St. James Town                               

In [23]:
df_restaurants[df_restaurants['Restaurant']=='Eat Canteen']

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Restaurant,Latitude,Longitude,Restaurant Category
94,"Adelaide, King, Richmond",43.650571,-79.384568,Eat Canteen,43.647288,-79.3817,Chinese Restaurant
135,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576,Eat Canteen,43.647288,-79.3817,Chinese Restaurant
159,"Commerce Court, Victoria Hotel",43.648198,-79.379817,Eat Canteen,43.647288,-79.3817,Chinese Restaurant
252,Stn A PO Boxes 25 The Esplanade,43.646435,-79.374846,Eat Canteen,43.647288,-79.3817,Chinese Restaurant
267,"First Canadian Place, Underground city",43.648429,-79.382280,Eat Canteen,43.647288,-79.3817,Chinese Restaurant


<a id='item5'></a>
## 5. Visualizating the neighbourhoods

### Map of all neighbourhoods and Chinese restaurants* in Toronto

**Chinese restaurants are within 500 m of the neighbourhood centers*

In [24]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [26]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, restaurant in zip(df_restaurants['Latitude'], df_restaurants['Longitude'], df_restaurants['Restaurant']):
    label = '{}'.format(restaurant)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
for lat, lng, neighborhood in zip(df_data['Latitude'], df_data['Longitude'], df_data['Neighbourhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#ff0000',
        fill=True,
        fill_color='#ff0000',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<a id='item6'></a>
## 6. Summary

Neighbourhoods in downtown Toronto can be clustered into 4 groups mainly due to their functionalities. 

The first group is charasterized by restaurants and coffee shops. The other groups have distinct functionalities like outdoor activities (group 2), Airport/travel services (group 3) and maybe residential area (group 4)

The grouping reflects the spatial distribution of the neighbourhoods where close neighbourhoods are grouped together.